<H3  style="font-family:verdana;"> 1- Collecting Products </H3>

<p  style="font-family:verdana;">The process below is repeated until we get all info needed. </p>

In [ ]:
#FIND ALL PRODUCTS FOR A GIVEN SEARCH-EXAMPLE SEPHORA COLLECTION

i=0
# Set up Selenium webdriver with Chrome
driver = webdriver.Chrome()

# Navigate to the URL
url = "https://www.amazon.com/s?k=sephora+collection&ref=nb_sb_noss"
driver.get(url)
l = list()
# Find all product links on the page
while True:
    # Wait for the next button to be clickable
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".s-pagination-container .s-pagination-next"))
    )
    
    # Find all product links on the page
    product_links = driver.find_elements(By.CSS_SELECTOR, "div[data-component-type='s-search-result'] h2 a")

    # Extract the href attribute of each link and print it
    for link in product_links:
        href = link.get_attribute("href")
        print(href)
        l.append(href)

    # Click the next button if it's not disabled
    if "s-pagination-disabled" in next_button.get_attribute("class"):
        break
    else:
        next_button.send_keys(Keys.ENTER)


# Quit the webdriver
driver.quit()

Get features : 'Product Name','Brand', 'Rating', 'Number of Ratings' , 'Price'

In [ ]:
# Launch a new instance of Chrome webdriver
driver = webdriver.Chrome()

l =list()
for url in data['links']:
        
    # Load the product page
    driver.get(url)
    try:
        # Wait for the product title element to load and extract the text
        product_title = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'productTitle')))
        product_name = product_title.text.strip()

        # Wait for the product rating element to load and extract the rating
        product_rating = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'span[id="acrPopover"]')))
        rating = product_rating.get_attribute('title')

        # Wait for the number of ratings element to load and extract the number of ratings
        num_ratings = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'acrCustomerReviewText')))
        num_ratings = num_ratings.text.split()[0]

        # Wait for the price element to load and extract the price
        price_whole= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-price-whole'))).text
        price_fraction= WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-price-fraction'))).text

        price = price_whole+'.'+price_fraction

        # Extract the brand name from the product name
        brand = product_name.split()[0]
        l.append({'Product Name':product_name,'Brand':brand, 'Rating': rating, 'Number of Ratings': num_ratings , 'Price':price})
    
        # Print the extracted information
        print('Product Name:', product_name)
        print('Brand:', brand)
        print('Rating:', rating)
        print('Number of Ratings:', num_ratings)
        print('Price:', price)
    
    except: 
        continue
# Close the browser window
driver.quit()


In [ ]:
#OUTPUT PRODUCTS
S_C_prod = pd.read_csv("C:/Users/Dell/Desktop/extract_info_sephora.csv")
MNYC_prod = pd.read_csv("C:/Users/Dell/Desktop/Products_maybelline.csv")
AMZ_prod = pd.read_csv("C:/Users/Dell/Desktop/extract_info_Amazon.csv")

<H3  style="font-family:verdana;"> 2- Collecting Reviews </H3>

Then we scrape reviews : access each product page and get the link to see all reviews

In [ ]:
#FOR EACH PRODUCT IN THE SEARCH PAGE WE FIND THE LINK TO SEE ALL REVIEWS
# Create a pandas DataFrame to store the review links

driver = webdriver.Chrome()
df_reviews = pd.DataFrame(columns=["link"])

# Loop through each product link and extract the review link
for link in l:
    # Navigate to the product page
    driver.get(link)

    # Scroll down to the "See all reviews" link and extract its href attribute
    see_all_reviews_link = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//a[contains(text(), 'See all reviews')]"))
    )
    review_link = see_all_reviews_link.get_attribute("href")

    # Add the review link to the DataFrame
    df_reviews = df_reviews.add({"link": review_link}, ignore_index=True)

# Quit the webdriver
driver.quit()

In [ ]:
#LOGIN INTO AMAZON ACCOUNT CLICK ON NEXT UNTIL IT IS DISABLED 
# Set up Chrome webdriver
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
driver = webdriver.Chrome(options=options)

# Set up the Chrome driver
#driver = webdriver.Chrome()
driver.get("https://www.amazon.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2FSephora-Collection-Limited-Holiday-Palette%2Fproduct-reviews%2FB08KZN91X5%2Fref%3Dnav_signin%3Fie%3DUTF8%26reviewerType%3Dall_reviews&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0&")

#Login
#EXCEPTION HANDLING #Wait until all page appears
username=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR , "input[name='email']")))

#make sure fields are clear when getting in 
username.clear()

#Fill with own info
username.send_keys("khayarat664@gmail.com")

login = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR , "input[type='submit']"))).click()


password=WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR , "input[name='password']")))

password.clear()

password.send_keys("13071999")

login = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR , "input[type='submit']"))).click()

data = pd.read_csv("C:/Users/Dell/Desktop/review_links_maybelline.csv")

reviews_list = []
for url in data['links']:    
    # Navigate to the URL
    driver.get(url)
    i = 0
    # Keep scrolling down and clicking on the "Next" button until there are no more reviews to load
    while True:
        try:
            name = driver.find_element(By.CSS_SELECTOR, ".product-title").text
            reviews = driver.find_elements(By.CSS_SELECTOR, ".review-text-content span")
            review_scores = driver.find_elements(By.CSS_SELECTOR, ".review-rating span:nth-child(1)")
            for review, review_score in zip(reviews, review_scores):
                print(review.text.strip())
                score = review_score.get_attribute("innerHTML").split(" ")[0]
                reviews_list.append({'Product Name': name, 'Review': review.text, 'Review Score': score})

            
            next_button = driver.find_element(By.XPATH, "//li[@class='a-last']/a")
            actions = ActionChains(driver)
            actions.move_to_element(next_button)
            actions.perform()
            time.sleep(2) # wait for the next page of reviews to load
            next_button.click()
   
        except StaleElementReferenceException:
            # Refresh the page or wait for the element to reload before retrying
            driver.refresh()
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//div[@data-hook='review']")))
            continue
        except:
            break

    # Close the webdriver
driver.quit()

#Convert the list of reviews, review scores, and product name into a Pandas dataframe
reviews_df = pd.DataFrame(reviews_list)

# Print the dataframe
print(reviews_df)

In [ ]:
#OUTPUT REVIEWS
S_C = pd.read_csv("C:/Users/Dell/Desktop/Sephora_reviews_raw.csv")
MNYC = pd.read_csv("C:/Users/Dell/Desktop/reviews_maybelline_raw.csv")
AMZ = pd.read_csv("C:/Users/Dell/Desktop/final_reviews2.csv")